В этом задании мы будем работать с таблицами, содержащими информацию о размещенных в автоклассифайде объявлениях и произведенных транзакциях.

Описание данных:

1. Размещенные объявления (`dp_live_adverts`) — таблица с оптимизированными данными `live_adverts`.

* `user_id` - ID пользователя в продукте (`global_id` в паспорте пользователя)

* `execution_date` — дата, на которую объявление находилось в live

* `advert_id` — уникальный номер объявления

* `created_at` — дата создания объявления

* `price` — стоимость авто в объявлении

* `region` — город, в котором продается авто

* `passport_id` — локальный ID пользователя

* `platform` — платформа, через которую было размещено объявление:

(a) android — мобильное приложение, android,

(b) ios — мобильное приложение, ios,

(c) desktop — подано через сайт,

(d) mobile — подано в мобильной версии сайта

* `auto_brand` — марка авто

* `auto_model` — модель авто

* `year` — год выпуска авто

* `userType` — тип кабинета пользователя на момент размещения объявления в live

 

2. Транзакции (`user_transactions`) 

* `payment_date` — дата платежа

* `type` — вид оплачиваемой услуги

* `sign` — "-1" указывает, что была приобретена услуга и списана оплата

* `amount` — количество списанных денег

* `advert_id` - id объявления к которому была применена платная услуга

* `tstamp` - время применения платной услуги

* `balance_consumption` - какая сумма списывалась с баланса личного счета

* `id` - ID транзакции

* `passport_id` — локальный ID пользователя

In [1]:
import pandas as pd
import numpy as np
from math import ceil
from clickhouse_driver import Client
from config import user_id, password
from statsmodels.stats import power

# работаем с двумя БД, поэтому создаем два подключения.
readDB = 'hardda'
readWriteDB = 'hardda_student_data'


# Создаем соединение с ClickHouse
client = Client(
    host='clickhouse.lab.karpov.courses',
    port=9000,
    user=user_id,
    password=password,
    database=readDB
)

def get_data(query):
    """
    Вытягивает данные из clickhouse в виде Dataframe
    
    query - запрос
    """
    result, columns = client.execute(query, with_column_types=True)
    return pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

In [2]:
query = """
    SELECT *
    FROM live_adverts
    LIMIT 10
"""
df = get_data(query)
df

,execution_date,advert_id,created_at,price,region,user_id,platform,auto_brand,auto_model,passport_id,year,userType
0,2022-02-01,130486371,2012-11-06 07:12:00,0,Москва,123578197,unknown,Unknown,Unknown model,123599132,0,0
1,2022-02-01,129236743,2012-11-06 07:31:13,0,Москва,123477706,unknown,Unknown,Unknown model,123493336,0,0
2,2022-02-01,130358537,2012-11-06 07:35:24,0,Москва,123799514,unknown,Unknown,Unknown model,123820472,0,0
3,2022-02-01,129230893,2012-11-06 08:01:17,0,Москва,123730252,unknown,Unknown,Unknown model,123751205,0,0
4,2022-02-01,169982253,2012-11-06 08:24:59,0,Москва,123653651,unknown,Unknown,Unknown model,123674596,0,0
5,2022-02-01,129615858,2012-11-06 08:39:13,0,Москва,123764303,unknown,Unknown,Unknown model,123785259,0,0
6,2022-02-01,128303169,2012-11-06 08:57:54,0,Москва,123548993,unknown,Unknown,Unknown model,123569924,0,0
7,2022-02-01,124841595,2012-11-06 08:59:45,0,Москва,123548993,unknown,Unknown,Unknown model,123569924,0,0
8,2022-02-01,124950166,2012-11-06 09:53:13,0,Москва,123597697,unknown,Unknown,Unknown model,123618634,0,0
9,2022-02-01,125269022,2012-11-06 15:08:04,9600,Москва,123569121,unknown,Unknown,Unknown model,123590055,0,0


### 1. Сделаем нужную выгрузку.

Подготовьте SQL запрос для клиентов, совершивших оплату в Москве в период с 11 по 16 октября 2022 г. за услугу "premium".

Пояснения для формирования SQL запроса:

1) Используйте таблицу `user_transactions`, чтобы оставить только тех пользователей, кто воспользовался платными услугами 'premium' в указанный период. Размер выручки от каждого покупателя услуги считается как сумма всех списанных денег (`amount`) при условии sign = -1 (т.е. деньги были списаны со счета покупателя).

2) Используйте таблицу `dp_live_adverts` для того, чтобы оставить объявления, размещенные в Москве. Обратите внимание, что одно и то же объявление может быть опубликовано в течение длительного периода (execution_date), а оплата за услугу может быть произведена в выборочные дни (`payment_date`). Учитывайте дату платежа и номер объявления при join'e таблиц.

3) Из запроса сформируйте датасет, включающий данные по выручке, которую приносит каждый пользователь (нужно взять `passport_id`!) в Москве за указанное время.

Чтобы убедиться, что вы все сделали правильно, ответьте на следующий вопрос.

Укажите длину полученной таблицы

In [ ]:
start_date = '2022-10-11'
end_date = '2022-10-16'

query = f"""
    SELECT
        u.type,
        u.passport_id,
        region,
        ABS(SUM(sign * amount)) AS revenue
    FROM user_transactions AS u
    LEFT JOIN dp_live_adverts la 
        ON la.advert_id::UInt32 = u.advert_id 
        AND la.execution_date = u.payment_date
    WHERE payment_date BETWEEN '{start_date}' AND '{end_date}'
        AND type = 'premium'
        AND sign = -1
        AND amount != 0
        AND region = 'Москва'
    GROUP BY u.type,
        u.passport_id,
        region
"""

df = get_data(query)
df

In [ ]:
df.shape[0]

### Задание 1. Размер выборки (1/3)

В продукте планируется внедрение нового функционала с целью улучшить привлекательность услуги "premium". Менеджеры предполагают, что это позволит увеличить среднюю выручку на 8% (поле `amount`).

Вам необходимо определить минимально необходимый размер выборки, чтобы статистически значимо задетектить эффект при увеличении выручки на 8% с учетом мощности теста в 80% и установленного уровня значимости alpha = 0.05.

Давайте решать эту задачу по шагам. В первую очередь, нам необходимо определить среднюю выручку и стандартное отклонение. И найти ожидаемую менеджером среднюю выручку с учетом ее увеличения на 8%.

Введите в поле значение выручки (с точностью до двух знаков), которую хотел бы достичь менеджер.

In [ ]:
effect_size = 0.08
test_power = 0.8
alpha = 0.05

In [ ]:
df['revenue'].describe()

In [ ]:
round(df['revenue'].describe()[1] * (1 + effect_size), 2)

### Задание 1. Размер выборки (2/3)
Найдите значение для стандартизированного эффекта (индекс Коэна).

В этом и следующих уроках для расчета стандартного отклонения используйте метод [std() из библиотеки pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.std.html) или функцию [std() из библиотеки numpy с параметром ddof=1](https://numpy.org/doc/stable/reference/generated/numpy.std.html)

Введите ответ в поле ниже с точностью до двух знаков в десятичной части.

Используйте точку для разделения целой и дробной части.

In [ ]:
mean_control = df['revenue'].mean()
mean_target = mean_control * (1 + effect_size)

std_control = df['revenue'].std()
effect_d = (mean_target - mean_control) / std_control
round(effect_d, 2)

### Задание 1. Размер выборки (3/3)
Используя рассчитанные значения, найдите минимально необходимый размер выборки, чтобы статистически значимо задетектить эффект.

Введите ответ в поле ниже. В ответе должно быть целое число.

In [ ]:
effect_size = 0.08
test_power = 0.8
alpha = 0.05

sample_size_n_target = ceil(power.TTestIndPower().solve_power(
    effect_size=effect_d,
    power=test_power,
    alpha=alpha,
    nobs1=None)
)

sample_size_n_target

### Задание 2. Размер выборки с новыми параметрами эксперимента
Менеджер предложил снизить ошибку II рода до 10%, при этом относительный эффект, который требуется задетектить, был установлен в 6%. Уровень значимости alpha остался прежним (0.05).

Оцените размер выборки, который вам потребуется, чтобы задетектить стат.значимый эффект от нововведения, если он присутствует.

Введите ответ в поле ниже. В ответе должно быть целое число.

Попробуйте изобразить графически, как будет меняться размер эффекта, который можно зафиксировать, в зависимости от размера выборки, выбранной мощности теста и уровня значимости

In [ ]:
effect_size = 0.06
test_power = 0.9
alpha = 0.05

mean_control = df['revenue'].mean()
mean_target = mean_control * (1 + effect_size)

std_control = df['revenue'].std()
effect_d = (mean_target - mean_control) / std_control

sample_size_n_target = ceil(power.TTestIndPower().solve_power(
    effect_size=effect_d,
    power=test_power,
    alpha=alpha,
    nobs1=None)
)


sample_size_n_target

### Задание 3. Размер эффекта

Имеется несбалансированная выборка из 100 тыс. пользователей, где соотношение между группами составляет 80:20 (на 20% пользователей распространяется нововведение, а на 80% нет: ratio = n2/n1 = 0.25). Дизайн теста предполагает вероятность в 5% совершить ошибку I рода и допущение в 20% совершить ошибку II рода.

Определите размер эффекта, который мы сможем задетектить при заданном дизайне эксперимента.

Введите ответ в поле ниже с точностью до ТРЕХ знаков в десятичной части.

Используйте точку для разделения целой и дробной части.



In [ ]:
test_power = 0.8
alpha = 0.05
sample_size = 100000
n1 = 80000
n2 = 20000

effect_size = round(power.TTestIndPower().solve_power(
    power=test_power,
    alpha=alpha,
    nobs1=n1,
    ratio=0.25)
, 3)

effect_size

### Задание 4. Эффект Коэна (1/2)
Максимальный размер в двух выборках, доступный вам, равен 100 тыс. пользователей. Определите, при каком соотношении между контрольной и тестовой группами (n2/n1) мы сможем задетектить минимальное значение размера эффекта (Cohen's D) при уровне значимости alpha = 0.05 и мощности теста = 80% и стат критерии t-test.

Для решения используйте симуляции, получая различные соотношения между двумя группами.

In [ ]:
max_population = 100000

n1 = np.arange(5000, 95000, 100)
effect_list = []
ratio_list = []

for i in range(len(n1)):
    effect_result = power.TTestIndPower().solve_power(effect_size = None,
                            power = 0.8,
                            nobs1 = n1[i],
                            ratio = (max_population - n1[i]) / n1[i],
                            alpha = 0.05)
    ratio = (100000 - n1[i]) / n1[i]
    ratio_list.append(ratio)
    effect_list.append(effect_result)

best_ratio = ratio_list[np.argmax(effect_list == np.min(effect_list))]

print('Минимальной ratio n2/n1 = {}'.format(best_ratio))

### Задание 4. Эффект Коэна (2/2)
Чему будет равна величина минимального эффекта - Cohen's D?

Введите ответ в поле ниже с точностью до ТРЕХ знаков в десятичной части.

Используйте точку для разделения целой и дробной части.

In [ ]:
round(np.min(effect_list), 3)